In [16]:
import sklearn
import pickle
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [50]:
stemmer = PorterStemmer()
stop_words = stopwords.words('english')
titles = ['biotechnology', 'business_economics', 'chemical_engineering', 'computer_science', 'electrical_engineering', 'geography', 'hospitality_&_tourism', 'law_school', 'medical_school', 'visual_design']

In [45]:
file = open("bernoulli_model_v1_c10_e230.pickle", "rb")
bernoulli = pickle.load(file)
file.close()
file = open("mnb_model_v1_c10_e230.pickle", "rb")
multinomial = pickle.load(file)
file.close()
file = open("logistic_model_v1_c10_e230.pickle", "rb")
logistic_regression = pickle.load(file)
file.close()

In [39]:
class VoteClassifier ():
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, prep_data):
        votes = [0] * len(self._classifiers[0]['clf'].classes_)
        for c in self._classifiers:
            v = c.predict_proba(prep_data)
            votes += v
        votes/=3
        return votes

In [135]:
s = "i dont want to be a game developer"

In [136]:
data = [s]
prep_data = []

for d in data:
    d = [stemmer.stem(dt).lower() for dt in d.split(' ')]
    d = [dt for dt in d if dt not in stop_words]
    prep_data.append(' '.join(d))

In [137]:
prep_data

['dont want game develop']

In [138]:
classifier = VoteClassifier(bernoulli,multinomial, logistic_regression)

In [139]:
res = classifier.classify(prep_data)
print(res)
print(titles[np.argmax(res)])

[[0.05304476 0.04458544 0.02724274 0.49026794 0.02319427 0.05243375
  0.04129497 0.06940807 0.16884211 0.02968594]]
computer_science


In [82]:
titles

['biotechnology',
 'business_economics',
 'chemical_engineering',
 'computer_science',
 'electrical_engineering',
 'geography',
 'hospitality_&_tourism',
 'law_school',
 'medical_school',
 'visual_design']